# Keras'ın Tanıtılması

In [ ]:
import tensorflow as tf

We'll load up the MNIST data set. In Keras, it's a little bit different - there are 60K training samples and 10K test samples. No "validation" samples.

MNIST veri setini yükleyeceğiz. Keras'ta biraz farklı - 60K eğitim örnekleri ve 10K test örnekleri var. "Doğrulama" örneği yok.

In [ ]:
(mnist_train_images, mnist_train_labels), (mnist_test_images, mnist_test_labels) = tf.keras.datasets.mnist.load_data()

Verileri Keras/TensorFlow'un beklediği biçime dönüştürmemiz gerekiyor. 
Görüntü verilerini float değerlere dönüştürdükten sonra 0-1 aralığına normalleştirmek için 255'e bölüyoruz.

In [ ]:
train_images = mnist_train_images.reshape(60000, 784)
test_images = mnist_test_images.reshape(10000, 784)

train_images = train_images.astype('float32')
test_images = test_images.astype('float32')

train_images /= 255
test_images /= 255

Şimdi 0-9 etiketlerini "one-hot" biçime dönüştüreceğiz.

In [ ]:
train_labels = tf.keras.utils.to_categorical(mnist_train_labels, 10)
test_labels = tf.keras.utils.to_categorical(mnist_test_labels, 10)

İyi göründüğünden emin olmak için eğitim görüntülerinden birine bir göz atalım:

In [ ]:
import matplotlib.pyplot as plt

def display_sample(num):
    #Print the one-hot array of this sample's label 
    print(train_labels[num])  
    #Print the label converted back to a number
    label = train_labels[num].argmax(axis=0)
    #Reshape the 768 values to a 28x28 image
    image = train_images[num].reshape([28,28])
    plt.title('Sample: %d  Label: %d' % (num, label))
    plt.imshow(image, cmap=plt.get_cmap('gray_r'))
    plt.show()
    
display_sample(1234)

Oluşturacağımız model. 784 öznitelikten oluşan giriş katmanı, 
512 düğümden oluşan bir ReLU katmanına beslenir ve daha sonra 
softmax uygulanmış olarak 10 düğüme gider.


In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(512, activation='relu', input_shape=(784,)))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

Ortaya çıkan modelin detaylı incelemesi:

In [ ]:
model.summary()

Optimizer ve kayıp fonksiyonu ayarlamak da aynı derecede basittir. Burada RMSProp optimizer'ı kullanacağız. 
Diğer seçenekler Adagrad, SGD, Adam, Adamax ve Naadam'dır. 
https://keras.io/optimizers/ adresine veya https://www.tensorflow.org/api_docs/python/tf/keras/optimizers bakın.

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(),
              metrics=['accuracy'])

Burada modelimizi 100 toplu örnek ile 10 tur (epoch) eğiteceğiz.

In [ ]:
history = model.fit(train_images, train_labels,
                    batch_size=100,
                    epochs=10,
                    verbose=2,
                    validation_data=(test_images, test_labels))

Model performansımıza bakalım

In [ ]:
score = model.evaluate(test_images, test_labels, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Yanlış tahmin sonuçlarını görselleştirelim. Bu model çok daha iyi olduğu için, bakılacak hataları bulmak için daha derine inmemiz gerekecek.

In [ ]:
for x in range(1000):
    test_image = test_images[x,:].reshape(1,784)
    predicted_cat = model.predict(test_image).argmax()
    label = test_labels[x].argmax()
    if (predicted_cat != label):
        plt.title('Prediction: %d Label: %d' % (predicted_cat, label))
        plt.imshow(test_image.reshape([28,28]), cmap=plt.get_cmap('gray_r'))
        plt.show()


Sorun yaşadıklarının çoğunu görebilirsiniz, bir insanın da sorun yaşayabileceği görüntüler!

## Egzersiz
Daha önce olduğu gibi, sonuçları iyileştirip iyileştiremeyeceğinize bakın! Daha fazla epoch(tur) çalıştırmak önemli ölçüde yardımcı olur mu? Farklı optimizerlar denemeye ne dersiniz?

Farklı topolojileri hızlı bir şekilde denemek için Keras'ın kullanım kolaylığından da yararlanabilirsiniz. Keras, ek bir katman ekledikleri ve aşırı uyumu önlemek için her adımda Dropout'u kullandıkları bir MNIST örneğini içerir, örneğin:

```
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))
```

Bunu yukarıdaki kodumuza uyarlamayı deneyin ve bir fark yaratıp yaratmadığına bakın.